# One-way Function
## Produced by: Ruben Girela Castellón

In [3]:
import numpy as np
import math
import random

# ========================================================================
# Functions of Modular Arithmetic
# ========================================================================

'''
función que calcula el maximo comun divisor y devuelve el mcd y u, v 
pertenecientes a numeros enteros.
'''
def mcd(x, y, u=[1,0], v=[0,1]):
    
    #Los pongo en valor absoluto
    dividendo = abs(x)
    divisor = abs(y)
            
    #calculo el cociente y el resto
    cociente, resto = divmod(dividendo, divisor)
    
    #si el resto es 0 termina y devuelve el cmd, u y v
    if(resto == 0): return divisor, u[-1], v[-1]
    
    '''
    en caso contrario copio la listas de u y v y añado el nuevo valor u y v
    a traves de la formula: u_i = u_(i-2) - q_i * u_(i-1), siendo q_i el cociente
    calculado.
    '''
    u1 = u.copy()
    u1.append(u1[-2]-cociente*u1[-1])
    v1 = v.copy()
    v1.append(v1[-2]-cociente*v1[-1])
    
    #y repetimos el proceso, hasta que el resto sea 0
    resultado = mcd(divisor,resto, u1, v1)
    
    '''
    esto se hace ya que es una función recursiva y tengo que ir pasando el 
    resultado en cada iteración recursiva.
    '''
    return resultado

#FUncion que dado un numero x, determina si x es probablemente primo usando el metodo Miller-Rabin
def isPrime(x):
    
    #por si el valor que recibe no es entero devuelve 0 (no es primo)
    if(type(x) != type(int())):
       return 0
    
    #si el numero x < 5
    if(x < 5):
        #y son 2 o 3 son probablemente primos
        if(x==2 or x ==3):
            return 1
        
        #en caso contrario no lo seran
        return 0
    
    #si el numero es par directamente no es primo
    if(x%2 == 0):
        return 0
    
    #para calcular s y u inicialmente valdran s = x - 1 y u = 0
    s = x-1
    u = 0
    
    #mientras s sea par
    while(s%2 == 0):
        #incremento u en 1
        u += 1
        #y divido s a la mitad entera
        s = s // 2
    
    #para saber si es primo o no inicialmente el resultado valdra 1
    result = 1
    
    #contador k que contara las veces que lo comprueba, en mi caso 10 como mucho
    k = 0
    
    #por defecto recorrera 10 veces
    maximo = 10
    
    '''
    pero si el numero es pequeño, por ejemplo 5, recorrera menos de 10 veces, 
    ya que estaríamos repitiendo numeros ya calculados anteriormente
    '''
    if(x-3 < 10):
        maximo = x-3
    
    #genero una lista de n numeros aleatorios entre [2, x-2] no repetidos
    lista = []#random.sample(range(2,x-1),maximo)
    
    
    #mientras el resultado es 1 o x-1 (-1) y no ha llegado al numero maximo de veces
    while((result == 1 or result == x-1) and k < maximo):
        
        #obtengo aleatoriamente otro numero
        #a = lista.pop()
        
        #obtengo un valor aleatorio
        a = random.randint(2,x-1)
        '''
        compruebo que no es repetido, si lo es calculo 
        otro valor, hasta que no sea un valor repetido
        '''
        while(a in lista):
            a = random.randint(2,x-1)
        #y lo añado a la lista
        lista.append(a)
        
        #contador que regulara el numero de calculos
        contador = 0
        
        #se hace el primer calculo a^s mod x
        result = pow(a,s,x)
        
        #si el resultado no es 1 o -1 (x-1) y el contador es < u
        while((result != 1 and result != x-1) and contador < u):
            #incremento el contador
            contador += 1
            #calculo a^s·2^k mod x
            result = pow(result,pow(2,contador),x)
            
            '''
                si a^(2^(k-1) * s) = 1, compruebo el siguiente:
                a^(2^(k) * s) es != de 1 o -1
            '''
            if((result == 1) and contador+1 < u):
                contador += 1
                result = pow(result, pow(2,contador),x)
           
        k += 1 #incremento el numero de veces
        
    #si ha llegado al numero maximo de veces y el resultado sigue siendo 1 o -1 (x-1)
    if(k== maximo and (result == 1 or result == x-1)):
        return 1 #es primo
    
    #en caso contrario no es primo
    return 0

#función que calcula a^(-1) mod b, para cualquier a, b enteros que sean primos relativos
def inversa(x, y):
    
    #para que los valores sean enteros
    x = int(x)
    y = int(y)
    
    #calculo el mcd, la v y u
    divisor, u, v = mcd(x,y)
    
    '''
    Compruebo solo el 1, ya que el divisor y el dividendo los convierto en valor 
    absoluto, con lo cual incluye tambien el -1.
    '''
    #si el divisor es 1 son primos relativos.
    if(divisor == 1):
        #calculo a^(-1) su inversa haciendo u mod b
        return u % y        
    
    #si no son primos relativos devuelvo -1 e imprimo un mensaje de error
    print("Error no son primos relativos, ya que ambos son divisibles por", divisor)
    return -1

#Aplicamos el simbolo jacobi (a/p) = 1   
def jacobi(a,p): 
    
    # le aplicamos a (a mod p)
    a = a % p
    
    #este nos indicará si existe un residuo cuadratico o no
    t=1
    
    #mientras a sea distinto de 0
    while(a != 0):
    
        #si a es un numero par
        while(a%2 == 0):
            
            #dividimos a/2
            a /= 2
            #y aplicamos p modulo 8
            r = p%8
        
            #si el resultado de p mod 8 es 5 o 3, cambiamos el signo de t
            if(r == 5 or r == 3):
                t = -t
    
        #intercambiamos los valores
        a, p = p, a
    
        #si a mod 4 y p mod 4 = 3 cambiamos el signo de t
        if(a%4 == 3 and p%4 == 3):
            t = -t
        
        #aplicamos a mod p
        a = a%p

    if(p == 1):
        #y devuelve 1 si existe y -1 en caso contrario
        return t
    else:
        return 0
    
'''
FUnción que implementa el algoritmo paso enano y paso gigante para el calculo
de algoritmos discretos en Zp. Es decir calcula x = log_b c mod p
'''
def enanoGigante(p, base, cuerpo):
    
    #si es primo
    if(isPrime(cuerpo)):
        
        '''
        c = cuerpo-1
        
        #metodo de Babylonia (para calcular la raiz cuadrada entera)
        
        while(c > (cuerpo-1)//c):
            c = (c+(cuerpo-1)//c)//2
        
        
        #El +1 es porque isqrt redondea hacia abajo (floor)
        c += 1
        print(f"c={c}")
        '''
        
        #calculo la raiz cuadrada entera
        c = math.isqrt(cuerpo-1) +1
        #El +1 es porque isqrt redondea hacia abajo (floor)
        
        '''
        genero 2 listas para us y vs, tal que
        u = {1,...,c}
        v = {0,...,c-1}
        '''
        us = range(1,c+1)
        vs = range(0,c)
        
        #calculo el paso enano
        #calculo el primero paso enano (b)
        r = p
        '''
        diccionario donde almacenare todos los resultados del paso enano [resultado] = v
        lo hago en un diccionario, para evitar añadir resultados repetitivos con distintos valores de v, 
        teniendo solo un unico resultado, con el valor v más pequeño.
        '''
        ls = {}
        
        #recorro todos los valores v de la lista
        for v in vs[1:]:
            
            '''
            si no esta en el diccionario lo meto, esto hace que evite valores repetidos, 
            haciendo que sea más eficiente.
            '''
            if(not r in ls):
                #guardo como key el resultado del paso enano y como valor v, que es la que nos interesa
                ls[r] = v-1
                
            #calculo el siguiente paso enano (b * a^v)
            r = (r*base)%cuerpo
            
        '''
        (el ultimo calculo no entra en el bucle)
        si no esta en el diccionario lo meto.
        '''
        if(not r in ls):
            #guardo como key el resultado del paso enano y como valor v, que es la que nos interesa
            ls[r] = vs[-1]
        
        #calculo el paso gigante
        Ls = []#lista que guardara los valores ya calculados para no comprobar valores repetidos
        
        #calculo el primero paso gigante (a^(c))
        r = pow(base,c,cuerpo)
        
        #recorro todos los valores de u en la lista
        for u in us[1:]:
            
            #si no esta en la lista
            if(not r in Ls):
                #la agrego
                Ls.append(r)
                #y compruebo si esta en el diccionario del paso enano
                if(r in ls):
                    #si esta obtengo v
                    v = ls[r]

                    #y calculo k = (c * u - v) y lo devuelvo
                    return int((c*(u-1)-v)%cuerpo)
                    #lo de u-1 es porque el primero ya se ha calculado fuera del bucle
            
            #calculo el nuevo valor del paso gigante (a^(c*u))
            r = (r*pow(base,c,cuerpo))%cuerpo

        #comprobamos el ultimo que se ha calculado, pero no se ha comparado
        #si no esta en la lista
        if(not r in Ls):
            #compruebo si esta en el diccionario del paso enano
            if(r in ls):
                #si esta obtengo v
                v = ls[r]

                #calculo k = (c * u - v) y lo devuelvo
                return int((c*(us[-1]-1)-v)%cuerpo)
                #lo de u-1 es porque el primero ya se ha calculado fuera del bucle

    '''
    si no es primo o no encuentra un L == l, siendo L un valor del calculo del paso gigante y
    l un valor del calculo del paso enano, no se puede aplicar este algoritmo.
    '''
    return -1

## Backpack function (Knapsack) and its inverse

We are going to use the case of the super-increasing sequence of positive numbers, for the **knappsack problem**.

To solve this problem, we use the Greedy algorithm, which consists of:

Given a sequence of values **a** and a maximum allowed size **n**, we are going to:

- Initialize a vector **b** to 0 with the same length as the sequence **a**.
- For i = {k, k-1, ..., 1, 0};  where **k** is the lenght of **a** - 1.
    - Compare if $a_i \leq n$:
        - Change the value $b_i=1$
        - $n = n - a_i$
    - And we repeat the process until n = 0:
        - return b

In [2]:
def mochilaGreedyCreciente(n,a):
    #creo el vector de soluciones
    r = np.zeros(len(a),dtype=int)
    
    #recorro los valores del conjunto a de forma decreciente
    for i in np.arange(len(a))[::-1]:
        
        #si el valor es <= a la capacidad máxima permitida
        if(a[i] <= n):
            
            #pongo un 1 de que se mete ese valor
            r[i] = 1
            # y le restamos a la capacidad máxima ese valor
            n -= a[i]
            
            #si la capacidad máxima es 0 termina y devuelve la solución
            if(n == 0):
                return r
    
    return r

Now with the algorithm Greedy we are going to implement the first practical public key criptosistem even proposed.

It was invented by **Merkle** and **Hellman**, shortly after **Diffie** and **Hellman** established the basic principles of public key criptography. 

Currently it is not used, since the keys obtained by disguising supercreasing sequences in this way are believed to be special and using the knapsack problem can be very easy.

### Cipher

Given a super-increasing sequence of positive numbers $(a_0, a_1, ..., a_k)$, where $\sum_{i=0}^{l}(a_i) < a_j; \thinspace j=l+1$ and the message **m** = a sequence of 0s and/or 1s of length **k**, where **k** is the length of the sequence **a**.

We generate 2 prime positive random numbers **n** and **u**, such that $ n > \sum_{i=0}^{k}a_i$ and **gcd(n, u) = 1**.

We generate with these values another sequence **a*** of the same length as **a**, which will contain the values $a^*_i = u * a_i \thinspace mod \thinspace n$.

And finally we generate $c = \sum_{i=0}^{k} a^*_i * m_i$, where **c** is the encrypted message.

### Decipher

Given a super-increasing sequence of positive numbers $(a_0, a_1, ..., a_k)$, the cipher message **c**, and the random values **n** and **u** generated in the cipher functión.

Let's decrypt the encrypted message. To do this we will calculate the inverse of **u** ($u^{-1}).

We calculate the value $b = c * u^{-1} \thinspace mod \thinspace n$, and we get the decrypted message using the Greedy algorithm.

With which we encrypt and decrypt using the private key (**a**, **u** and **n**), and the public key (**a***)


In [3]:
#recibe una secuencia de numeros creciente
def mochilaCifrado(a, m):
    
    n = 0
    #escojo un valor aleatorio n > a la suma de todas las secuencias de a
    #en mi caso, le he puesto un tope de que sea como mucho 2*sum(a) y que sea primo
    while(not isPrime(n)):
        n = np.random.randint(low=sum(a)+1,high=2*sum(a))
        
    u = 0
    #lo mismo con u escogemos un valor aleatorio primo entre 0 y sum(a)
    while(not isPrime(u)):
        u = np.random.randint(low=0,high=sum(a))
    
    #comprobamos que el maximo comun divisor es 1 de esos 2 valores
    if(mcd(n,u)[0] == 1):
        
        #si lo es pasamos a y m a un array
        a = np.array(a)
        m = np.array(m)
        
        #y calculamos la secuencia a_i* = a_i * u mod n
        a_star = (a*u)%n
        
        '''
        y posteriormente aplicamos la llave publica al mensaje m y obtenemos el 
        mensaje cifrado.
        '''
        c = np.sum(a_star * m)
        
        return c, u, n
    
    #en caso contrario error
    return -1

def mochilaDescifrado(c, u, n, a):
    
    #calculamos la inversa de u^1 mod n
    u_inv = inversa(u,n)
    
    #calculamos el valor b, para posteriormente obtener el mensaje descifrado
    b = (c * u_inv)%n
    
    #aplicamos el algoritmo greedy para obtener nuestro mensaje descifrado
    return mochilaGreedyCreciente(b, a)

Example:

I use a function called **generadorListaCreciente** that generates the super-increasing sequence.

Using the ASCII table to generate our message, and let's assume the space (' ') is the code 7.

![Si no visualizas la imagen, es porque la carpeta imagenes no se encuentra en el mismo directorio que este cuaderno.](imgs/codigo-ascii.jpg "ASCII table")

We encrypt the message (77, 105, 7, 109, 101, 110, 115, 97, 106, 101), obtaining our encrypted message:

In [4]:
#generador de secuencias super-creciente
def generadorListaCreciente(n, umbral):
    
    lista = []
    
    for i in np.arange(n):
        
        if(i == 0):
            lista.append(np.random.randint(0,umbral))
        else:
            lista.append(np.random.randint(sum(lista)+1,sum(lista)+umbral))
    
    return lista

lista = generadorListaCreciente(8,11)

#Suponemos que el espacio es representado con el codigo ASCII 7
mensaje = [77, 105, 7, 109, 101, 110, 115, 97, 106, 101]
#convertimos el mensaje en binario
mensaje_bits = []
for i in mensaje:
    m = []
    for e in bin(i)[2:]:
        m.append(int(e))
        
    if(len(m)<8):
        m = [0]*(8-len(m))+m
    mensaje_bits.append(m)

print(f'Mensaje: {mensaje}\n')
print(f'Mensaje binario: {mensaje_bits}\n')

#ciframos el mensaje
c = [0]*len(mensaje)
u = [0]*len(mensaje)
n = [0]*len(mensaje)
e = 0

for i in mensaje_bits:
    c[e], u[e], n[e] = mochilaCifrado(lista, i)
    e += 1

print(f'Mensaje cifrado {c}\n')

#desciframos el mensaje en bits
mensaje_descifrado_bits = []

for i in np.arange(len(c)):
    
    mensaje_descifrado_bits.append(
        mochilaDescifrado(c[i], u[i], n[i], lista).tolist())
    
#y traducimos el mensaje binario a decimal
mensaje_descifrado = []

for i in mensaje_descifrado_bits:
    
    mensaje_descifrado.append(int('0b'+''.join(map(str,i)), 2))

print('Mensaje descifrado:')
print(f'{mensaje_descifrado} == Mi mensaje (traducido a la tabla ASCII)')



Mensaje: [77, 105, 7, 109, 101, 110, 115, 97, 106, 101]

Mensaje binario: [[0, 1, 0, 0, 1, 1, 0, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 1, 1, 1, 0], [0, 1, 1, 1, 0, 0, 1, 1], [0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 0, 1]]

Mensaje cifrado [6652, 5858, 3114, 7939, 2936, 6733, 3425, 2532, 4083, 6888]

Mensaje descifrado:
[77, 105, 7, 109, 101, 110, 115, 97, 106, 101] == Mi mensaje (traducido a la tabla ASCII)


## Compute the inverse, such that $\alpha^x = \thinspace$ birth date

For this we need a **p** pseudo-prime $\geq$ than the identity number **id = 75569264**. For this I choose $p \geq id$ randomly and make it prime **p** and also $\frac{p-1}{2}$.

With these data we calculate an $2 \leq \alpha \leq p-2$ in $\mathbb Z_p$, such that $\frac{\alpha}{p} = -1$ (using jacobi).

With **p**, $\alpha$ and the number of birth date **num = AAAAMMDD**, we calculate the inverse of the birth date:

To calculate it, just apply $log_\alpha (num) \thinspace mod \thinspace p$ (Baby-step giant-step).

And we obtain a unique solution that when applying $x^\alpha \thinspace mod \thinspace p = num$.


In [5]:
def inversoNaci(p, alpha, num):
    
    #calculamos la unica solución que nos da
    x = enanoGigante(num, alpha, p)
    
    #y comprobamos que al aplicar alpha^x mod p = fecha de nacimiento
    if(pow(alpha, x, p) == num):
        return x

    return -1

#identidad
id = 75569264

#peseudo-primo >= id
p = np.random.randint(id, 9999999999)

while(not isPrime(p) or not isPrime((p-1)//2)):
    p = np.random.randint(1, 9999999999)
    #print('not prime',p)

print(f'pseudo-primo: {p}')

#obtenemos un numero alpha sea 2 <= alpha <= p-2, de tal forma que jacobi de -1
alpha = 2
while(jacobi(alpha,p) != -1 and alpha < p-1):
    if(alpha%2 != 0):
        alpha += 2
    else:
        alpha += 1

print(f'alpha: {alpha}')
print(f'{alpha}^2 != 1; {pow(alpha,2,p)}')

num = 19950619
x = inversoNaci(p, alpha, num)
print(f'{num}^-1 = {x}')
print(f'{alpha}^{x} = {pow(alpha,x,p)}')

pseudo-primo: 7528494899
alpha: 2
2^2 != 1; 4
19950619^-1 = 6577594796
2^6577594796 = 19950619


## Break n to get p and q

Given the Rabin function $f(x) = x^2$. Let $n = 48478872564493742276963$, I know that $f(12) = 144 = f(37659670402359614687722)$.

With these data we are going to calculate **p** and **q**, in such a way that $p*q=n$:

It is assumed that $a^2 - b^2 \thinspace mod \thinspace n = 0$, in this case it is verified that the substraction is 0, $12^2 - 37659670402359614687722^2 \thinspace mod \thinspace n = 0$.

So $(a+b)(a-b) = 0$, so $a+b = 0$, but this is not the case, since $12 + 37659670402359614687722 \thinspace mod \thinspace n = 37659670402359614687734$.

So we can break by applying the  **gcd(a+b, n) = p**, getting one of the coefficients **p**, and the other coefficient as $q = \frac{n}{p}$.

In [6]:
def rompePQ(n, a, b):
    
    print(f'Se supone que a^2 - b^2 mod n = 0; lo comprobamos y efectivamente es {(pow(a,2,n)-pow(b,2,n))%n}')
    
    print(f'Con lo cual (a+b)(a-b) = 0; a+b = 0; en este caso no lo es: {(a+b)%n}')
    
    print('Entonces podemos romperlo:')
    
    x = mcd((a+b)%n,n)
    
    p = x[0]
    q = n//p
    
    return p, q
    


n = 48478872564493742276963
a = 12
b = 37659670402359614687722

p, q = rompePQ(n, a, b)
print(f'p = {p}, q = {q}, n = {p*q}')

Se supone que a^2 - b^2 mod n = 0; lo comprobamos y efectivamente es 0
Con lo cual (a+b)(a-b) = 0; a+b = 0; en este caso no lo es: 37659670402359614687734
Entonces podemos romperlo:
p = 303948303229, q = 159497098847, n = 48478872564493742276963


## H summary function

Given 2 variables $a_0$ and $a_1$, 2 arbitrary squares modulo $n = 48478872564493742276963$.

Let's implement a hash function:

$$h: \mathbb Z_2 \times (\mathbb Z_n)^* \rightarrow (\mathbb Z_n)^*, h(b,x) = x^2·a_0^b·a_1^{1-b}$$

Using the **Merkle-Damgard construction**, we implement the digest function as a compression function.

![Si no visualizas la imagen, es porque la carpeta imagenes no se encuentra en el mismo directorio que este cuaderno.](imgs/merkle_damgard_hash.png "Merkle-Damgard construction")

(Imagen obtenida de: https://hmong.es/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction)

Where it initially receives a message block and a initial vector, it apllies the hash function **h(b,x)** and the result substitutes the value of the ial vector and the process is repeated with the next block, until obtaining our hash in the last calculation.

In [7]:
#función h(x,b) = x^2 * a0^b * a1^1-b
def h(a0, a1, x, b):
    
    return (pow(x,2,n)*pow(a0,b,n)*pow(a1,1-b, n))%n
    

def MD(m, vi, a0, a1):
    #guardamos en v el vector inicial
    v = vi
    
    #recorremos el mensaje
    for i in m:
        #y aplicamos el hash
        v = h(a0, a1, v, i)
        
    return v

n = 48478872564493742276963

#selecciono 2 cuadrados albitrarios aleatoriamente modulo n
a = random.choice(range(2,len(str(n))))%n
a0 = pow(a,2,n)
a = random.choice(range(2,len(str(n))))%n
a1 = pow(a,2,n)

#por si escoge el mismo numero aleatoriamente (es poco probable)
while(a0 == a1):
    a = random.choice(range(2,len(str(n))))%n
    a1 = pow(a,2,n)

print(f'Generamos a0 = {a0} y a1 = {a1} (2 cuadrados aleatorios)')

m = [1, 1, 0, 0, 1, 0, 1, 0]

# vi (vecor inicial) pertenece a Zn^* = Zn \ {0} (sin 0)
vi = 3

print(f'Dado un vector inicial {vi} y el conjunto de bloques del mensaje {m}')
mi_hash = MD(m, vi, a0, a1)

print(f'Obtenemos el hash: {mi_hash}')

Generamos a0 = 64 y a1 = 324 (2 cuadrados aleatorios)
Dado un vector inicial 3 y el conjunto de bloques del mensaje [1, 1, 0, 0, 1, 0, 1, 0]
Obtenemos el hash: 27965853428435799360610


## RSA function

We are going to implement the RSA function, which consists of given 2 distant large primes **p** and **q** and a message **m**, we will encrypt the message using:

$$n = pq, \phi(n) = (p-1)(q-1)$$

We take a relative prime number and quotient **e** with $\phi(n)$ and the gcd(e, $\phi(n)$) = 1.

And therefore it will have an inverse module $\phi(n)$

With these data we have the *public key* **n** abd **e** and *the private key* **d**.

With that we can encrypt and decrypt:

- **Encrypt**: $m^e \thinspace mod \thinspace n$
- **Decrypt**: $m^d \thinspace mod \thinspace n$

In this case we obtain **p** and **q** as follows:

- **p**: We take a $p \geq dni$, being the smallest possible prime.
- **q**: We take a $q \geq date\_of\_birth$, being the smallest possible prime.




In [8]:
#Algoritmo RSA
def RSA(m, p, q, cifrado=True):
    
    #calculamos phi(n) como el producto de (p - 1) * (q - 1)
    n1 = (p-1)*(q-1)
    #obtenemos n = p * q
    n = p*q
    
    #cogemos un primo relativo a n1, tociente (para cifrar)
    e = n1
    #mientras el mcd(e, n1) no sea 1 o e == n1
    while(mcd(e,n1)[0] != 1 or e == n1):
        #incrementamos e
        e +=1
        #y buscamos un primo
        while(not isPrime(e)):
            if(e%2 != 0):
                e += 2
            else:
                e += 1
    
    #calculamos su inverso: (para descifrar)
    e_inverso = inversa(e, n1)
    
    #si quiere cifrar un mensaje
    if(cifrado):
        return pow(m,e,n)

    #si quiere descifrar un mensaje
    return pow(m,e_inverso,n)


dni = 75569264
#cogemos un p > = al dni, siendo el menor primo posible
p = dni

while(not isPrime(p)):
    if(p%2 != 0):
        p += 2
    else:
        p += 1
        
print(f'p = {p}')

birthday = 19950619
#cogemos un q >= a la fecha de nacimiento, siendo el menor primo posible
q = birthday

while(not isPrime(q)):
    
   if(q%2 != 0):
       q += 2
   else:
       q += 1
    
print(f'q = {q}')

p = 75569287
q = 19950647


Examples:

In [9]:
m = 128

c = RSA(m, p, q)

print(f'Mensaje {m} se cifra con RSA y obtenemos c = {c}')

m = RSA(c, p, q, False)

print(f'Desciframos el mensaje cifrado con RSA y obtenemos m = {m}')

Mensaje 128 se cifra con RSA y obtenemos c = 158773878519538
Desciframos el mensaje cifrado con RSA y obtenemos m = 128


In [10]:
c = 1234567890
print(f'Desciframos el mensaje {c} y obtenemos el mensaje m = {RSA(c,p,q,False)}')

Desciframos el mensaje 1234567890 y obtenemos el mensaje m = 1065954479971057


## Find q and p

Given an $n = 50000000385000000551$ and we know $e = 5$ and its inverse $d = 10000000074000000101$, we know both the public key and the private key.

Then we can finf **p** and **q** in n as follows:

1. We calculate $(e·d)-1 = 2^a · b$, where **b** is an odd number.
2. Then we randomly compute a value $1 < x < n$ and calculate the **gcd(x,n)**:
    - If **gcd(x,n) != 1**:
        - We have a factor **p**, and calculate the factor $q = n \thinspace // \thinspace p$.
    - If **gcd(x,n) == 1**:
        3. We calculate $y = x^b \thinspace mod \thinspace n$
        4. If **y** is not 1 nor -1:
            5. We store **y** in an auxiliary variable $z = y$
            6. We update $y = y^2$ and repeat point D
        7. If **y = 1**: (We have found a **z**, such that $z^2 \thinspace mod \thinspace n = 1$)
            - Calculate $p = gcd(n, z + 1)$
            - Calculate $q = gcd(n, z - 1)$
8. In other case repeat point 2

Comparing this algorithm with the Miller-Rabin algorithm, they are quite similar, sice both search for prime numbers, calculating in both algorithms $x - 1 = 2^a · b$, but with the difference that Miller-Rabin has a maximum number of times (10) and this algorithm does not stop until it finds it.

Also comparing our algorithm with the **rompePQ** of exercise 3, we see that it is somthing similar too, since both try to find a greatest common divisor between **x** and **n**, different from 1 and -1, finding the factors **p** and **q**, but with the difference that in the exercise 3 is more direct, since it finds it intantly, while this algorithm repeats the process **k** times, being slower.

In [11]:
def findQP(n, e, d):
    
    
    while(True):
        #obtenemos a y b, tal que (e*d)-1 = 2^a * b
        b = e*d - 1
        a=0
        while(b%2==0):
            b = b//2
            a += 1
        
        #seleccionamos un valor x entre (0,n), ambos no incluidos
        x = random.randrange(1,n-1) #para valores muy grandes

        #si el maximo comun divisor de x y n es 1
        if(mcd(x,n)[0] == 1):
            #obtenemos un y = x^b mod n
            y = pow(x,b,n)
            
            find = False
            
            #Si y es != +-1
            while(y != 1 and y != n-1):
                #guardamos en una variable auxiliar el valor de y
                z = y
                
                #Y actualizamos el valor y = y^2 mod n
                y = pow(y,2,n)

                #si y = 1, hemos encontrado un z, tal que z^2 mod n = 1
                if(y == 1):
                    
                    #con lo cual gcd(n, z + 1) y gcd(n, z - 1) son los factores primos de n
                    p = mcd(n, z + 1)[0]
                    q = mcd(n, z - 1)[0]
                    
                    return p, q
                
        else:# en caso contrario hemos encontrado un factor
            p = x
            #y calculamos el segundo factor como una división entera
            q = n // x
            return p, q

Example:

In [12]:
n = 50000000385000000551
e = 5
d = 10000000074000000101

p, q = findQP(n, e, d)

print(f'Partiendo de los valores n = {n}, e = {e} y d = {d}.')
print(f'Podemos obtener los valores p = {p} y q = {q}.')
print(f'De tal forma que {p} * {q} = {p*q}')

Partiendo de los valores n = 50000000385000000551, e = 5 y d = 10000000074000000101.
Podemos obtener los valores p = 5000000029 y q = 10000000019.
De tal forma que 5000000029 * 10000000019 = 50000000385000000551


## Digital signature and signature verification, with RSA

To perform the digital signature, we have to encrypt the message firt, in my case I encrypt the message using RSA, RSA needs the **public key (e, n)**, the **private key (d)** and the message (m).

The public key is generated:

- $n = p·q$, where p = first found prime $\geq$ dni, and q = first found prime $\geq$ date of birth.
- e = a relative prime (quotient) to $\phi(n) = (p-1)·(q-1)$ and the gcd(e, $\phi(n)$) = 1

The private key is generated $d = e^{-1} \thinspace mod \thinspace \phi(n)$

Once the message is encrypted, we will need to apply a hash function to said encrypted message, in order to apply the digital asignature. In my case I use the hash function already implement in exercise 4.

The hash need: 

- The message in bits
- A initial vector
- And 2 square random numbers differents from each other, in a range between [2, n].

Once we have the hash, we can now apply the digital asignature, which will need the hash and the private key.

The RSA digital asignature consists of $f = c^d \thinspace mod \thinspace n$, where **c** is the encrypted message and **d** is the private key.

with that we can now send our encrypted message, the hash and the asignature. In such a way that the recipient, when receiving it, can verify the veracity of the message, verifying the authenticity of the asignature.

To verify it, we need the signature, the hash and the public key. And we can verify if the message is authentic or has been modified by checking it $f^e \thinspace mod \thinspace n = hash$.

In [4]:
#Algoritmo RSA
def RSA2(m, n, e, d, cifrado=True):
    
    #si quiere cifrar un mensaje
    if(cifrado):
        return pow(m,e,n)

    #si quiere descifrar un mensaje
    return pow(m,e_inverso,n)


def firmaRSA(c, d, n):
    #para generar la firma f = m^d mod n
    f = pow(c,d,n)
    
    return f

def verificaciónFirmaRSA(f, c, e, n):#, d
    
    '''
    print(f)
    print(pow(c,d,n))
    print(pow(f,e,n))
    print(pow(pow(c,d,n),e,n))
    print(c)
    '''
    
    #si f^e = (m^d)^e mod n = c mod n, es decir calcular la veracidad de la firma
    if(pow(f,e,n) == c):
        return 1
    
    return 0

Example: (transmitter)

In [25]:
#cojo p y q con el dni y la fecha de nacimiento usado en el ejercicio 5
#cogemos un p > = al dni, siendo el menor primo posible
p = 75569264

while(not isPrime(p)):
    if(p%2 != 0):
        p += 2
    else:
        p += 1
        
print(f'p = {p}')

birthday = 19950619
#cogemos un q >= a la fecha de nacimiento, siendo el menor primo posible
q = birthday

while(not isPrime(q)):
    
   if(q%2 != 0):
       q += 2
   else:
       q += 1
    
print(f'q = {q}')

#calculamos phi(n) como el producto de (p - 1) * (q - 1)
n1 = (p-1)*(q-1)
#obtenemos n = p * q
n = p*q

print(f'phi(n) = {n1} and n = {n}')

#cogemos un primo relativo a n1, tociente (para cifrar)
e = n1
#mientras el mcd(e, n1) no sea 1 o e == n1
while(mcd(e,n1)[0] != 1 or e == n1):
    #incrementamos e
    e +=1
    #y buscamos un primo
    while(not isPrime(e)):
        if(e%2 != 0):
            e += 2
        else:
            e += 1

#calculamos su inverso: (para descifrar)
e_inverso = inversa(e, n1)

print(f'Private key: d = {e_inverso} and public key: e = {e}')

m = 7
#cifro el mensaje
c = RSA2(m, n, e, e_inverso)
print(f'Mensaje cifrado (RSA): {c}')

#Aplicamos el resumen (utilizo la función resumen ya creado)

#Para ello selecciono 2 cuadrados albitrarios aleatoriamente modulo n
a = random.choice(range(2,len(str(n))))%n
a0 = pow(a,2,n)
a = random.choice(range(2,len(str(n))))%n
a1 = pow(a,2,n)

#por si escoge el mismo numero aleatoriamente (es poco probable)
while(a0 == a1):
    a = random.choice(range(2,len(str(n))))%n
    a1 = pow(a,2,n)

m_bits = list(map(int,bin(m)[2:]))

# vi pertenece a Zn^* = Zn \ {0} (sin 0)
vi = 3

print(f'Dado un vector inicial {vi} y el conjunto de bloques del mensaje {m} = {m_bits}')
mi_hash = MD(m_bits, vi, a0, a1)

print(f'Hash: {mi_hash}')

#Una vez obtenido el mensaje cifrado firmamos
f = firmaRSA(mi_hash, e_inverso, n)

print(f'Firma: {f}')

p = 75569287
q = 19950647
phi(n) = 1507656073458756 and n = 1507656168978689
Private key: d = 1114354489078211 and public key: e = 1507656073458779
Mensaje cifrado (RSA): 264904610774926
Dado un vector inicial 3 y el conjunto de bloques del mensaje 7 = [1, 1, 1]
Hash: 836674568108910
Firma: 1395922393193744


Example: (receiver)

In [26]:
print(f'Recibe:\n mensaje cifrado = {c}, hash = {mi_hash} y la firma = {f}')
print(f'Y con su clave publica: e = {e}, n = {n}, comprobamos la autenticidad de la firma')

v = verificaciónFirmaRSA(f, mi_hash, e, n)#, e_inverso

print(f'¿Es autentica la firma {f}? {bool(v)}')

print(f'Desciframos el mensaje: {RSA2(c, n, e, e_inverso, False)}')

Recibe:
 mensaje cifrado = 264904610774926, hash = 836674568108910 y la firma = 1395922393193744
Y con su clave publica: e = 1507656073458779, n = 1507656168978689, comprobamos la autenticidad de la firma
¿Es autentica la firma 1395922393193744? True
Desciframos el mensaje: 7


Example where the message is modify:

In [22]:
print('Si el mensaje ha sido modificado:')
mod_hash = mi_hash + 1
print(f'hash = {mi_hash}, hash modificado = {mod_hash}')

v = verificaciónFirmaRSA(f, mod_hash, e, n)

print(f'¿Es autentica la firma {f}? {bool(v)}')

print('\nY lo mismo ocurriria si solo modificamos la firma:')

mod_f = f + 1
print(f'firma = {f}, firma modificada = {mod_f}')

v = verificaciónFirmaRSA(mod_f, mi_hash, e, n)

print(f'¿Es autentica la firma {mod_f}? {bool(v)}')

Si el mensaje ha sido modificado:
hash = 147419183679003, hash modificado = 147419183679004
¿Es autentica la firma 1066012824201346? False

Y lo mismo ocurriria si solo modificamos la firma:
firma = 1066012824201346, firma modificada = 1066012824201347
¿Es autentica la firma 1066012824201347? False


## Biliography

- https://www.thinglink.com/scene/533827640969134081
- https://es.wikipedia.org/wiki/N%C3%BAmero_pseudoprimo
- https://cacr.uwaterloo.ca/hac/about/chap2.pdf
- https://hmong.es/wiki/Initialization_vector
- https://numerentur.org/funcion-merkle-damgard/
- https://hmong.es/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Licencia Creative Commons Atribución-NoComercial-SinDerivadas 4.0 Internacional</a>.